In [31]:
import gc
from copy import deepcopy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras import backend as K

from libs.util import random_mask
from libs.pconv_model_first_resid import PConvUnet
# from libs.pconv_model_UNet import PConvUnet
from libs.properties import properties

# Settings
MAX_BATCH_SIZE = 32

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
properties_dict = properties()
length = properties_dict["length"]

In [33]:
matrix_df = pd.read_csv('./data/trafficV_M.csv', index_col=0, parse_dates=True)

In [34]:
# matrix_df = np.array(matrix_df)

# print(matrix_df.shape)
# print("%d bytes" % (matrix_df.size * matrix_df.itemsize))
# print(np.isnan(matrix_df).sum())

In [35]:
# # 单独测试week_ago, mintue_ago
# week_delta = pd.Timedelta(1, unit='W')
# minute_delta = pd.Timedelta(15, unit='m')


# channel_num = 3
# smooth_time = channel_num-1
# set_up_time = week_delta

# train_df = matrix_df.truncate(before=matrix_df.index.min() + set_up_time)
# train_week_ago_df = matrix_df.loc[train_df.index - week_delta]
# train_minute_ago_df = matrix_df.loc[train_df.index - minute_delta]

# train_df = np.array(train_df).reshape(-1, length, length, 1)
# train_week_ago_df = np.array(train_week_ago_df).reshape(-1, length, length, 1)
# train_minute_ago_df = np.array(train_minute_ago_df).reshape(-1, length, length, 1)


# train_array = np.concatenate((train_df, train_minute_ago_df, train_week_ago_df), axis=3)
# X_train, X_val = train_test_split(train_array, test_size = 0.1, random_state=42)

In [36]:
def createTrainArray(week_history_num=0, minute_history_num=0):
    week_delta_list = [pd.Timedelta(i+1, unit='W') for i in range(week_history_num)]
    minute_delta_list = [pd.Timedelta((i+1)*15, unit='m') for i in range(minute_history_num)]
    delta_list = week_delta_list+minute_delta_list
    print(delta_list)
    
    set_up_time = pd.Timedelta(week_history_num, unit='W')
    train_df = matrix_df.truncate(before=matrix_df.index.min() + set_up_time)
    
    train_ago_array_tuple = tuple([np.array(matrix_df.loc[train_df.index - i]).reshape(-1, length, length, 1) for i in delta_list])
    train_df = np.array(train_df).reshape(-1, length, length, 1)
    train_array = np.concatenate((train_df,)+train_ago_array_tuple, axis=3)
    print(train_array.shape)
    return train_array

In [37]:
week_history_num = 1
minute_history_num = 1

channel_num = week_history_num +minute_history_num +1
smooth_time = channel_num-1

train_array = createTrainArray(week_history_num, minute_history_num)
X_train, X_val = train_test_split(train_array, test_size = 0.1, random_state=42)

[Timedelta('7 days 00:00:00'), Timedelta('0 days 00:15:00')]
(16704, 32, 32, 3)


In [38]:
epoch_steps = X_train.shape[0] // MAX_BATCH_SIZE
val_steps = X_val.shape[0] // MAX_BATCH_SIZE
epoch_steps, val_steps

(469, 52)

In [39]:
class DataGenerator(ImageDataGenerator):
    def flow(self, X, *args, **kwargs):
        while True:
            
            # Get augmentend image samples
            ori = next(super().flow(X, *args, **kwargs))
    
            # Get masks for each image sample
            mask = np.stack([random_mask(ori.shape[1], ori.shape[2], size=0.1, channels=channel_num, smooth_time=smooth_time) for _ in range(ori.shape[0])], axis=0)

            # Apply masks to all image sample
            masked = deepcopy(ori)
#             print(masked.shape)
            masked_mean = masked[mask==1].mean()
            masked[mask==0] = masked_mean

            # Yield ([ori, masl],  ori) training batches
#             print(masked.shape, ori.shape)
            gc.collect()
            yield [masked, mask], ori[:,:,:,:1]
            
train_datagen = DataGenerator()
train_generator = train_datagen.flow(
    X_train, batch_size=MAX_BATCH_SIZE
)

# Create validation generator
val_datagen = DataGenerator()
val_generator = val_datagen.flow(
    X_val, batch_size=MAX_BATCH_SIZE
)

In [40]:
# np.array(next(train_generator)[1])[:,:,:,:1].shape

In [41]:
model = PConvUnet(img_rows=length, img_cols=length, channels=channel_num)
optimizer = model.get_optimizer()
def scheduler(epoch):
    if epoch % 3 == 0 and epoch != 0:
        lr = K.get_value(optimizer.lr)
#         if lr>0:
        if lr>0.0001:
            if epoch%6==0:
                K.set_value(optimizer.lr, lr * 0.5)
            else:
                K.set_value(optimizer.lr, lr * 0.7)
        print(K.get_value(optimizer.lr))
    return K.get_value(optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=1, mode='auto')

In [42]:
model.fit(
    train_generator, 
    validation_data=val_generator,
    steps_per_epoch = epoch_steps,
    validation_steps = val_steps,
    epochs = 10,
    callbacks=[reduce_lr]
)

Epoch 1/1
469/469 [==============================] - 184s 391ms/step - loss: 191729.6646 - val_loss: 84860.7022
Epoch 2/2
469/469 [==============================] - 181s 386ms/step - loss: 55356.0371 - val_loss: 57218.7720
Epoch 3/3
469/469 [==============================] - 176s 374ms/step - loss: 45888.9830 - val_loss: 203225.2870
Epoch 4/4
0.0034999999
469/469 [==============================] - 174s 372ms/step - loss: 37141.9557 - val_loss: 39440.1926
Epoch 5/5
469/469 [==============================] - 185s 395ms/step - loss: 37431.5096 - val_loss: 34222.5630
Epoch 6/6
469/469 [==============================] - 196s 418ms/step - loss: 34815.8151 - val_loss: 37653.6553
Epoch 7/7
0.0017499999
469/469 [==============================] - 205s 438ms/step - loss: 28913.2113 - val_loss: 44361.0327
Epoch 8/8
469/469 [==============================] - 198s 421ms/step - loss: 28689.8631 - val_loss: 32198.9102
Epoch 9/9
469/469 [==============================] - 206s 439ms/step - loss: 28031.9

In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 32, 32, 32)   896         input_5[0][0]                    
__________________________________________________________________________________________________
activation_43 (Activation)      (None, 32, 32, 32)   0           conv2d_39[0][0]                  
__________________________________________________________________________________________________
average_pooling2d_7 (AveragePoo (None, 16, 16, 32)   0           activation_43[0][0]              
__________________________________________________________________________________________________
batch_norm

In [44]:
list = []
for _ in range(20):
    list.append(model.evaluate_generator(val_generator, 1))
list
# model.evaluate_generator(val_generator, 100)

[25329.5546875,
 23260.11328125,
 25005.0234375,
 17110.798828125,
 28538.640625,
 21236.263671875,
 27065.431640625,
 25957.4609375,
 36996.0625,
 22413.37109375,
 29478.91015625,
 44412.96484375,
 29648.85546875,
 31601.576171875,
 26256.375,
 28283.34375,
 30239.09375,
 26424.453125,
 33294.765625,
 28220.994140625]

In [45]:
# import random
# test_num = random.randint(0, 200)
# test = deepcopy(X_train[test_num,np.newaxis,:])

# test_mask = random_mask(test.shape[1], test.shape[2], size=0.1, channels=channel_num, smooth_time=smooth_time)
# test_mask = test_mask[np.newaxis,:]

# test_mask[0,:,:,0].shape
# test[test_mask==0] = test.mean()

# # test_mask.shape
# # plt.imshow(test[0,:,:,0]*255)
# test_res = model.predict([test, test_mask])
# # np.sum((test-test_res)**2)
# np.sum((test[0,:,:,0][test_mask[0,:,:,0] == 0] - test_res[0,:,:,0][test_mask[0,:,:,0] == 0])**2)